In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Extracting ./mnist/data\train-images-idx3-ubyte.gz
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz


In [6]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])  # 28x28 차원. 2차원 중 첫번째. 마지막 차원 1은 특징의 개수 = 회색조 이미지라 채널이 한개
Y = tf.placeholder(tf.float32, [None, 10])  # 출력값이 10개인 분류. 2차원 중 두번째
keep_prob = tf.placeholder(tf.float32)

In [7]:
# 3x3 크기의 컨벌루션 계층 생성
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')   # 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직이는 옵션. 이미지 테두리까지 조금 더 정확하게 평가 가능
L1 = tf.nn.relu(L1)

# 입력층 X와 첫 번째 계층의 가중치 W1을 가지고 오른쪽과 아래쪽으로 한 칸씩 움직이는 32개의 커널을 가진 컨볼류션 계층을 만들겠다는 코드.



# 고수준 API 적용 시 아래와 같이 줄어 듬
# L1 = tf.layers.conv2d(X, 32, [3, 3])
# L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])

In [8]:
# 풀링 계층 생성
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 앞서 만든 컨볼루션 계층을 입력층으로, 커널 크기를 2x2로 하는 풀링계층 생성. Strides 값은 슬라이딩 시 두 칸씩 움직이겠다는 옵션

In [11]:
# 두 번째 계층 생성

W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')

# 3x3 크기 커널 64개의 컨볼루션 계층, 2x2크기의 풀링 계층.
# W2 변수의 구성은 [3,3,32,64]. 32는 앞서 구성한 첫 번째 컨볼루션 계층의 커널 개수. -> 출력층의 개수이며 또한 첫 번째 컨볼루션 계층이 찾아낸 이미지의 특징 개수

In [13]:
# 추출한 특징들을 이용해 10개의 분류를 만들어내는 계층 구성

W3 = tf.Variable(tf.random_normal([7 * 7* 64,256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])  # 차원 을 줄임.1차원 계층으로 만들고. 
L3 = tf.matmul(L3, W3) # 이 배열 전체를 최종 출력값의 중간단계인 256개의 뉴런으로 연결하는 신경망 생성
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

# 인접한 계층의 모든 뉴런과 상호 연결 -> 풀리 커넥티드 레이어
# 그리고 과적합을 막아주는 드롭아웃 사용.



# 고수준 API 적용 시 아래와 같이 줄어 듬
# L3 = tf.contrib.layers.faltten(L2)
# L3 = tf.layers.dense(L1, 256, activation=tf.nn.relu)

# 이렇게 tf.layers 등의 고급 API 사용 시, 활성화 함수나 컨볼루션 계층을 만들기 위한 나머지 수치들을 알아서 계산하고 적용해 줌.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
# 은닉층인 L3의 출력값 256개를 받아 최종 출력값인 0~9 레이블을 갖는 10개의 출력값 생성
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

# 손실함수 생성 및 AdamOptimizere
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [18]:
# 학습을 시키고 결과를 확인하는 코드
### 신경망 모델 학습 ###

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1),
         'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.351
Epoch: 0002 Avg. cost = 0.109
Epoch: 0003 Avg. cost = 0.076
Epoch: 0004 Avg. cost = 0.060
Epoch: 0005 Avg. cost = 0.047
Epoch: 0006 Avg. cost = 0.041
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.031
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.023
Epoch: 0011 Avg. cost = 0.021
Epoch: 0012 Avg. cost = 0.019
Epoch: 0013 Avg. cost = 0.018
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.015
최적화 완료!


In [20]:
### 결과 확인

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape( -1, 28, 28, 1), 
                                               Y: mnist.test.labels, keep_prob: 1}))


# 뉴런과 계층 수를 늘리거나 하이퍼 파라미터들을 변경하면서 정확도 향상 실험 해보면 좋다.

정확도: 0.9894
